# HCAN 361 Final Project: Spring 2021

## Physical Access to Airborne Infection Rooms in US Hospitals: An Exploratory Analysis and Retrospective Assessment of the Implications with COVID Pandemic

In [1]:
use HCAN361_FinalProj

select *
from dbo.AS2019_DataDictionary
where [Item No#] is not null and [Field Description] like '%airborne%' 
order by [Item No#]

--First we identified the provider ids (hospitals) that were either classified as an Airborne Hospitals and the number of rooms that hospital had to offer -- 

SELECT A.ID
      ,cast(A.LAT as float) as LAT
	  ,cast(A.LONG as float) as LONG
	  ,A.MNAME
      ,C.County_Name,C.State_Name,C.HRSA_RURAL_DESC,('Z'+C.STATEFP+C.COUNTYFP) as [County_GEOID_Z]
	  ,case when C.[Median_ADI_NatRank]<=25 then '02: Upper Class'
	   when (C.[Median_ADI_NatRank]>25 and C.[Median_ADI_NatRank]<=75) then '01: Middle Class'
	   when C.[Median_ADI_NatRank]>75 then '00: Lower Class'
	   else '99: Not Ranked' end as [ADI]
	  ,D.AIRBROOM
into #hospitals_With
FROM 
dbo.AS19DEM as A
left join
[dbo].[Provider_Census_Tract] as B
	on A.ID=B.Provider_ID
left join 
[dbo].[US_Census_Tract_Charac] as C
	on B.GEOID_Data=C.GEOID_Data
inner join
dbo.AS19SVC1 as D
	on A.ID=D.ID
where ((D.AIRBHOS is not null) and (D.AIRBROOM is not null))
and LAT is not null
order by C.State_Name;
--note two facilities (ID='6740273' and '6740909')are clipped from analyses due to geocoding problems

select top 10 * from #hospitals_With;

select distinct *
into [HCAN361_03].dbo.AirRooms_Group3
from #hospitals_With;

alter table [HCAN361_03].dbo.AirRooms_Group3
alter column ID varchar(30) not null;

alter table [HCAN361_03].dbo.AirRooms_Group3
add constraint pk_air_Grp3 primary key(ID);



(4 rows affected)

(3415 rows affected)

(10 rows affected)

(3415 rows affected)

Total execution time: 00:00:00.612

Item No#,FieldName,Estimated,Length,Start,End,Field Description,Survey Question,Access Table,Notes
283,AIRBHOS,N,1,2729,2729,Airborne infection isolation room - hospital,C.21.,as19svc1,"1=yes, 0=no"
284,AIRBSYS,N,1,2730,2730,Airborne infection isolation room - health system,C.21.,as19svc1,"1=yes, 0=no"
285,AIRBVEN,N,1,2731,2731,Airborne infection isolation room - joint venture,C.21.,as19svc1,"1=yes, 0=no"
286,AIRBROOM,N,4,2732,2735,Number of airborne infection isolation rooms,C.21.,as19svc1,Number of rooms


ID,LAT,LONG,MNAME,County_Name,State_Name,HRSA_RURAL_DESC,County_GEOID_Z,ADI,AIRBROOM
6232810,40.9613,-78.9997,Punxsutawney Area Hospital,Jefferson,Pennsylvania,01: Rural,Z42065,01: Middle Class,3
6232980,41.4001,-75.6463,Geisinger-Community Medical Center,Lackawanna,Pennsylvania,00: Urban,Z42069,01: Middle Class,6
6233010,41.4105,-75.6507,Moses Taylor Hospital,Lackawanna,Pennsylvania,00: Urban,Z42069,01: Middle Class,12
6231120,40.0472,-76.3041,Penn Medicine Lancaster General Hospital,Lancaster,Pennsylvania,00: Urban,Z42071,01: Middle Class,57
6239183,40.1317,-76.3103,UPMC Lititz,Lancaster,Pennsylvania,00: Urban,Z42071,01: Middle Class,12
6230745,40.1872,-76.1824,WellSpan Ephrata Community Hospital,Lancaster,Pennsylvania,00: Urban,Z42071,01: Middle Class,12
6231510,41.0136,-80.3539,UPMC Jameson,Lawrence,Pennsylvania,01: Rural,Z42073,00: Lower Class,4
6230019,40.3371,-76.4175,WellSpan Good Samaritan Hospital,Lebanon,Pennsylvania,00: Urban,Z42075,01: Middle Class,12
6230040,40.6084,-75.4684,St. Luke's Sacred Heart Campus,Lehigh,Pennsylvania,00: Urban,Z42077,00: Lower Class,6
6230013,40.5669,-75.519,Lehigh Valley Hospital,Lehigh,Pennsylvania,00: Urban,Z42077,02: Upper Class,128


In [2]:
--Next, we built population tables to help us determine county level rates of air isolation rooms per people
select A.[GEOID_Data], ('Z'+A.STATEFP+A.COUNTYFP) as GEOID_County_Z,
[State_Name],[County_Name], [HRSA_RURAL_DESC],
case when [Median_ADI_NatRank]<=25 then '02: Upper Class'
when ([Median_ADI_NatRank]>25 and [Median_ADI_NatRank]<=75) then '01: Middle Class'
when [Median_ADI_NatRank]>75 then '00: Lower Class'
else '99: Not Ranked' end as [ADI],B.B01001e1 as [Total_Pop]
into #Population
from [dbo].[US_Census_Tract_Charac] A
left join
[dbo].[Census_ACS5Yr_2018] B
	on A.GEOID_Data=B.GEOID_Data;

select top 10 *
from #Population


(74001 rows affected)

(10 rows affected)

Total execution time: 00:00:00.324

GEOID_Data,GEOID_County_Z,State_Name,County_Name,HRSA_RURAL_DESC,ADI,Total_Pop
14000US01001020500,Z01001,Alabama,Autauga,00: Urban,01: Middle Class,9883.00000000
14000US01001021000,Z01001,Alabama,Autauga,00: Urban,01: Middle Class,2550.00000000
14000US01003010701,Z01003,Alabama,Baldwin,00: Urban,02: Upper Class,8324.00000000
14000US01003010903,Z01003,Alabama,Baldwin,00: Urban,01: Middle Class,5789.00000000
14000US01003011102,Z01003,Alabama,Baldwin,00: Urban,01: Middle Class,4385.00000000
14000US01003011405,Z01003,Alabama,Baldwin,00: Urban,02: Upper Class,4647.00000000
14000US01003011407,Z01003,Alabama,Baldwin,01: Rural,01: Middle Class,4984.00000000
14000US01005950800,Z01005,Alabama,Barbour,01: Rural,01: Middle Class,2347.00000000
14000US01005950900,Z01005,Alabama,Barbour,01: Rural,01: Middle Class,3994.00000000
14000US01009050500,Z01009,Alabama,Blount,00: Urban,01: Middle Class,7455.00000000


In [3]:
--Next we identified the correlating variables from the Travel_matrix to understand the distances per census tract to the nearst hospital

Select Distinct A.*
                ,ISNULL(B.Closest_Hospital,361) AS Closest_Hospital
				,ISNULL(C.AIRBROOM_60Min,0) AS AIRBROOM_60Min
into #Air_TravelMap
from
--Population of interest
#Population A
left join
--Find closest hospital when all are open
(
select GEOID_Data, MIN(TOTAL_TRAVELTIME) AS Closest_Hospital
from 
[dbo].[Travel_Matrix]
where Provider_ID in
(select ID from #hospitals_With)
Group by GEOID_Data
) B
	on A.GEOID_Data=B.GEOID_Data
left join
--Find available Nurse FTE within 30 minutes
(
select A.GEOID_Data, SUM(B.AIRBROOM) AS AIRBROOM_60Min
from 
[dbo].[Travel_Matrix] as A
left join
#hospitals_With B
	on A.Provider_ID=B.ID
where A.Provider_ID in
(select ID from #hospitals_With)
and A.TOTAL_TRAVELTIME<=60
Group by GEOID_Data
) C
	on A.GEOID_Data=C.GEOID_Data
order by A.GEOID_Data;


select top 10 * from #Air_TravelMap;

select distinct *
into [HCAN361_03].dbo.Air_Travel_Group3
from #Air_TravelMap;

alter table [HCAN361_03].dbo.Air_Travel_Group3
alter column GEOID_Data varchar(30) not null;

alter table [HCAN361_03].dbo.Air_Travel_Group3
add constraint pk_trvl_Grp3 primary key(GEOID_Data);

(74001 rows affected)

(10 rows affected)

(74001 rows affected)

Total execution time: 00:00:21.002

GEOID_Data,GEOID_County_Z,State_Name,County_Name,HRSA_RURAL_DESC,ADI,Total_Pop,Closest_Hospital,AIRBROOM_60Min
14000US37049960200,Z37049,North Carolina,Craven,00: Urban,00: Lower Class,8057.00000000,27.25552428000185,53
14000US37049960402,Z37049,North Carolina,Craven,00: Urban,01: Middle Class,4215.00000000,8.461779627353216,66
14000US37049960403,Z37049,North Carolina,Craven,00: Urban,01: Middle Class,4081.00000000,9.959876939914603,66
14000US37049960404,Z37049,North Carolina,Craven,00: Urban,01: Middle Class,2999.00000000,14.008071355868058,66
14000US37049960900,Z37049,North Carolina,Craven,00: Urban,01: Middle Class,2100.00000000,6.775670073239803,66
14000US37049961201,Z37049,North Carolina,Craven,01: Rural,00: Lower Class,2895.00000000,35.552849859547685,33
14000US37049961301,Z37049,North Carolina,Craven,01: Rural,01: Middle Class,3873.00000000,22.56762063326011,33
14000US37049961302,Z37049,North Carolina,Craven,01: Rural,01: Middle Class,5513.00000000,27.22678442383218,33
14000US37051000800,Z37051,North Carolina,Cumberland,00: Urban,01: Middle Class,2611.00000000,3.290643308105689,80
14000US37051000900,Z37051,North Carolina,Cumberland,00: Urban,01: Middle Class,4778.00000000,5.782664762832561,74


In [4]:
--Air isolation room per 10,000 people within each county

Select Distinct
 A.GEOID_County_Z,A.State_Name,A.County_Name
,A.Total_Pop,isnull(B.AIRBROOM,0) as AIRBROOM
,(isnull(B.AIRBROOM,0)/(Total_Pop*1.00))*10000 as [Rooms per 10K people]
into #CountyMap
From
(select GEOID_County_Z,State_Name,County_Name, sum(Total_Pop) as Total_Pop 
from #Air_TravelMap
Group by GEOID_County_Z,State_Name,County_Name) as A
left Join
(select State_Name,County_Name, County_GEOID_Z,sum(AIRBROOM) as AIRBROOM
from #hospitals_With
Group by State_Name,County_Name, County_GEOID_Z) as B
	on A.GEOID_County_Z=B.County_GEOID_Z
order by A.GEOID_County_Z


select top 10 * from #CountyMap;

select distinct *
into [HCAN361_03].dbo.CountyMap_Group3
from #CountyMap;

alter table [HCAN361_03].dbo.CountyMap_Group3
alter column GEOID_County_Z varchar(30) not null;

alter table [HCAN361_03].dbo.CountyMap_Group3
add constraint pk_cnty_Grp3 primary key(GEOID_County_Z);

(3220 rows affected)

(10 rows affected)

(3220 rows affected)

Total execution time: 00:00:00.386

GEOID_County_Z,State_Name,County_Name,Total_Pop,AIRBROOM,Rooms per 10K people
Z01001,Alabama,Autauga,55200.00000000,6,1.0869565217391304
Z01011,Alabama,Bullock,10352.00000000,0,0
Z01021,Alabama,Chilton,43930.00000000,3,0.6829046209879354
Z01039,Alabama,Covington,37351.00000000,1,0.2677304489839629
Z01045,Alabama,Dale,49255.00000000,0,0
Z01047,Alabama,Dallas,40029.00000000,0,0
Z01057,Alabama,Fayette,16585.00000000,3,1.808863430810974
Z01063,Alabama,Greene,8426.00000000,0,0
Z01065,Alabama,Hale,14887.00000000,0,0
Z01071,Alabama,Jackson,52094.00000000,0,0


In [5]:
--Table 1: Airborne Room Stats per Each State: Median Closest Facility

select *
from
(
select distinct 
'United States' as State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital)OVER () AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital end) OVER () AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital end) OVER () AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='02: Upper Class' then Closest_Hospital end) OVER () AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='01: Middle Class' then Closest_Hospital end) OVER () AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='00: Lower Class' then Closest_Hospital end) OVER () AS [Low SES]
from #Air_TravelMap

--below code concatenates (or stacks tables on top of each other)
union all

select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY Closest_Hospital)OVER (PARTITION BY State_Name) AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then Closest_Hospital end) OVER (PARTITION BY State_Name) AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then Closest_Hospital end) OVER (PARTITION BY State_Name) AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='02: Upper Class' then Closest_Hospital end) OVER (PARTITION BY State_Name) AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='01: Middle Class' then Closest_Hospital end) OVER (PARTITION BY State_Name) AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='00: Lower Class' then Closest_Hospital end) OVER (PARTITION BY State_Name) AS [Low SES]
from #Air_TravelMap) as A
order by State_Name;


(53 rows affected)

Total execution time: 00:00:07.301

State_Name,Statewide,Urban,Rural,High SES,Middle SES,Low SES
Alabama,17.788264129772354,14.26137613893247,30.286509767227024,14.740998170861692,17.254627898503045,23.152902149309572
Alaska,50.10803922478232,8.551320410617702,361,19.214447714497286,61.69867227800455,58.70888772373145
Arizona,9.718165398166148,9.099652006027101,52.107900149912965,10.381337646676824,9.365967190108753,9.76278189780125
Arkansas,14.364774159775564,10.743243218406999,19.65331514567244,11.327327037182751,14.502583293362964,14.86182156650747
California,9.738945263347784,9.387325200822556,31.955101400781594,9.470928830267207,10.074012818913639,9.818201207939502
Colorado,10.31316329483245,9.604124103980642,33.85194484286155,11.074122946801337,10.005633028762002,7.5930266378148685
Connecticut,11.780825684366441,11.453521121370411,16.05074299988178,14.224850573638054,9.300944783614474,4.887659011619254
Delaware,12.529651240864364,11.071542070922083,24.496023813159827,11.929101615215671,12.876565987351931,9.75320005979638
District of Columbia,8.769429455348696,8.769429455348696,NULL,7.409095564602547,12.550334477323743,17.749679546408867
Florida,11.928331893194807,11.631501588733817,24.809443179336508,12.329534786462453,11.984305960644939,11.230832359468701


In [6]:
--Table 2: Airborne Room Stats per Each State: Median Rooms Within an Hour

select *
from
(
select distinct 
'United States' as State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY AIRBROOM_60Min)OVER () AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then AIRBROOM_60Min end) OVER () AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then AIRBROOM_60Min end) OVER () AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='02: Upper Class' then AIRBROOM_60Min end) OVER () AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='01: Middle Class' then AIRBROOM_60Min end) OVER () AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='00: Lower Class' then AIRBROOM_60Min end) OVER () AS [Low SES]
from #Air_TravelMap

--below code concatenates (or stacks tables on top of each other)
union all

select distinct
 State_Name
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY AIRBROOM_60Min)OVER (PARTITION BY State_Name) AS Statewide   
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC='00: Urban' then AIRBROOM_60Min end) OVER (PARTITION BY State_Name) AS Urban
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when HRSA_RURAL_DESC<>'00: Urban' then AIRBROOM_60Min end) OVER (PARTITION BY State_Name) AS Rural
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='02: Upper Class' then AIRBROOM_60Min end) OVER (PARTITION BY State_Name) AS [High SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='01: Middle Class' then AIRBROOM_60Min end) OVER (PARTITION BY State_Name) AS [Middle SES]
,PERCENTILE_CONT(0.5) WITHIN GROUP (ORDER BY case when ADI='00: Lower Class' then AIRBROOM_60Min end) OVER (PARTITION BY State_Name) AS [Low SES]
from #Air_TravelMap) as A
order by State_Name;


(53 rows affected)

Total execution time: 00:00:06.566

State_Name,Statewide,Urban,Rural,High SES,Middle SES,Low SES
Alabama,39,49,25,267,45,32
Alaska,5,61,0,61,0,34
Arizona,537,553,5,546,496,480
Arkansas,108,171,21,171.5,115,61
California,415,424,13,458,250,233
Colorado,632,632,4,632,409.5,141
Connecticut,528,541,241.5,523,528,544
Delaware,549,634,75,436,595,905
District of Columbia,1024,1024,NULL,1024,1031,860
Florida,492,515.5,57,532,506,470.5
